# Model

In [2]:
!aws s3 ls


2023-08-29 23:43:16 sagemaker-studio-692960231031-wo7kgoszj2g
2023-08-29 23:50:01 sagemaker-us-east-1-692960231031
2023-08-30 00:34:21 vad49
2023-09-16 16:02:10 vad49-labdata


In [3]:
!aws s3 ls s3://project17-bucket-alex/stories-and-books-nlp/


2023-11-18 18:55:58          0 _SUCCESS
2023-11-18 18:55:54    2866925 part-00000-e5360052-4545-4fe2-9b12-7c9ce44bf3ec-c000.snappy.parquet
2023-11-18 18:55:55    2564497 part-00001-e5360052-4545-4fe2-9b12-7c9ce44bf3ec-c000.snappy.parquet
2023-11-18 18:55:56      26078 part-00035-e5360052-4545-4fe2-9b12-7c9ce44bf3ec-c000.snappy.parquet
2023-11-18 18:55:56      12157 part-00051-e5360052-4545-4fe2-9b12-7c9ce44bf3ec-c000.snappy.parquet
2023-11-18 18:55:57      28228 part-00053-e5360052-4545-4fe2-9b12-7c9ce44bf3ec-c000.snappy.parquet
2023-11-18 18:55:57      30806 part-00059-e5360052-4545-4fe2-9b12-7c9ce44bf3ec-c000.snappy.parquet


In [4]:
if True is False: # set to true only for the first un
    # Setup - Run only once per Kernel App
    %conda install openjdk -y

    # install PySpark
    %pip install pyspark==3.2.0 s3fs pyarrow torch

    # restart kernel
    from IPython.core.display import HTML
    HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [5]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import col, length, isnan, when, count, regexp_extract, weekofyear, hour, avg, to_date, unix_timestamp, lit, corr


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 150) 
#pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)


In [6]:
# Import pyspark and build Spark session

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    #.config("spark-jars-packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3")\
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-97fc5c93-cd0f-4192-9454-122ddf1f555f;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 423ms :: artifacts dl 19ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

3.2.0


In [7]:
# read in the data

all_model_text = spark.read.parquet("s3a://project17-bucket-alex/stories-and-books-nlp/")

#display(f"shape: ({all_model_text.count()}, {len(all_model_text.columns)})")
all_model_text.show()


23/11/18 19:34:37 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------------------+--------------------+--------------------+
|                text|       custom_tokens|         text_as_int|
+--------------------+--------------------+--------------------+
|       Metamorphosis|     [metamorphosis]|[29, 21, 36, 17, ...|
|      by Franz Kafka|[by,  , franz,  ,...|[18, 41, 1, 22, 3...|
|Translated by Dav...|[translated,  , b...|[36, 34, 17, 30, ...|
|                   I|                 [i]|                [25]|
|One morning, when...|[one,  , morning,...|[31, 30, 21, 1, 2...|
|“What’s happened ...|[whats,  , happen...|[39, 24, 17, 36, ...|
|Gregor then turne...|[gregor,  , then,...|[23, 34, 21, 23, ...|
|“Oh, God”, he tho...|[oh, ,,  , god, ,...|[31, 24, 3, 1, 23...|
|He slid back into...|[he,  , slid,  , ...|[24, 21, 1, 35, 2...|
|And he looked ove...|[and,  , he,  , l...|[17, 30, 20, 1, 2...|
|He was still hurr...|[he,  , was,  , s...|[24, 21, 1, 39, 1...|
|The first thing h...|[the,  , first,  ...|[36, 24, 21, 1, 2...|
|It was a simple m...|[it

In [8]:
# run on a sample

all_model_text = all_model_text.sample(False, 0.1)



In [9]:
# helper functions
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [10]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
import string
from tqdm import tqdm
#from queue import PriorityQueue
#from Levenshtein import distance 


# Collect all sequences into one long sequence
flattened_seq = all_model_text.select("text_as_int").rdd.flatMap(lambda x: x[0]).collect()

# Convert the flattened sequence into a PyTorch tensor
char_dataset = torch.tensor(flattened_seq, dtype=torch.long)



In [11]:
from pyspark.sql.functions import col, explode


# Collect all unique tokens
all_tokens = all_model_text.select(explode(col("custom_tokens"))).distinct().rdd.flatMap(lambda x: x).collect()

# Create the vocabulary from these tokens
vocab = sorted(set(all_tokens))


In [12]:
# %% [markdown]
# #### Efficiently load and batch the dataset for training using a `DataLoader`. Make sure to reserve some of the data for validation and testing. Describe how you handle batching and sequence lengths.
# 


In [13]:



# %%
# define sequences
seq_length = 100
examples_per_epoch = len(char_dataset) // (seq_length + 1)


# create the dataset
#char_dataset = torch.tensor(text_as_int, dtype=torch.long)

sequences = []
targets = []
for i in range(0, len(char_dataset) - seq_length, seq_length):
    sequences.append(char_dataset[i:i+seq_length])
    targets.append(char_dataset[i+1:i+seq_length+1])

dataset = TensorDataset(torch.stack(sequences), torch.stack(targets))



# split it
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])


# turn into dataloader
batch_size = 64

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)



In [14]:
# %% [markdown]
# Batching and Sequence Length
# 
# - 64 sequences per batch
# - 100 characters per sequence

# %% [markdown]
# #### Define your RNN model. Discuss the number of layers, hidden units, and the type of RNN cells you use. What is the total number of parameters in your model? Explain the rationale behind your architectural choices.
# 

In [15]:

# %%
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers):
        super(RNN, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
        
        # output
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = self.fc(x)
        
        return x


In [16]:

# %%

# hyperparameters
vocab_size = len(vocab)
embedding_dim = 128
hidden_dim = 256
n_layers = 2


# instantiate
model = RNN(vocab_size, embedding_dim, hidden_dim, n_layers)


display(model)


total_params = count_parameters(model)
print(f'total number of parameters: {total_params}')


RNN(
  (embedding): Embedding(9239, 128)
  (rnn): LSTM(128, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=9239, bias=True)
)

total number of parameters: 4478615


In [17]:
# %% [markdown]
# Choices
# 
# - RNN Cells: using LSTM cells over basic RNN cells since they are more resistant to the vanishing gradient problem and can capture long-range dependencies better.
# - Layers: I set this to 2, as it worked well for purposes of this model - more layers could improve the model but also be more expensive and make it more likely to overfit.
# - Hidden Units: I set to 256 hidden units for the same reasons as above.
# - Embedding Dimension: 128 for the same reason as above.
# 
# Note: total number of parameters printed above

# %% [markdown]
# #### Write the training loop.
# #### Monitor and report on the training progress by tracking the loss. After training, evaluate the model's performance using a suitable evaluation metric (e.g., perplexity) on a validation dataset or a held-out portion of the training data. Discuss the results.
# #### Specify the hyperparameters (for example, model hyperparameters, as well as sampling size and beam width from below) used in your model. Find suitable settings using the validation split.
# 

In [18]:




# %%
# training params
learning_rate = 0.01
num_epochs = 1


# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



# training loop
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    
    model.train()
    running_train_loss = 0.0
    
    # progress bar
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    
    
    
    # loop through data loader
    for batch_idx, (data, target) in loop:
        
        optimizer.zero_grad()
        output = model(data)
        output = output.permute(0, 2, 1)
        
        loss = criterion(output, target)
        loss.backward()
        
        optimizer.step()
        
        # acummulate loss
        running_train_loss += loss.item()
        
        # update tqdm bar
        loop.set_description(f"epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())
    
    # get avg loss for the epoch
    train_loss = running_train_loss / len(train_loader)
    train_losses.append(train_loss)
    train_perplexity = torch.exp(torch.tensor(train_loss))
    
    
    # validate
    model.eval()
    running_val_loss = 0.0
    val_loop = tqdm(enumerate(val_loader), total=len(val_loader), leave=False)
    
    with torch.no_grad():
        
        # loop through validation
        for batch_idx, (data, target) in val_loop:
            
            output = model(data)
            output = output.permute(0, 2, 1)
            
            loss = criterion(output, target)
            running_val_loss += loss.item()
            
            
            # tqdm
            val_loop.set_description(f"validation epoch [{epoch+1}/{num_epochs}]")
            val_loop.set_postfix(val_loss=loss.item())
    
    
    # get avg loss for the epoch
    val_loss = running_val_loss / len(val_loader)
    val_losses.append(val_loss)
    val_perplexity = torch.exp(torch.tensor(val_loss))
    
    # report
    print(f"epoch {epoch+1}/{num_epochs}, train loss: {train_loss:.4f}, train perplexity: {train_perplexity:.4f}, val loss: {val_loss:.4f}, val perplexity: {val_perplexity:.4f}")



# eval
model.eval()
test_loss = 0.0

with torch.no_grad():
    
    for batch_idx, (data, target) in enumerate(test_loader):
        
        output = model(data)
        output = output.permute(0, 2, 1)
        loss = criterion(output, target)
        test_loss += loss.item()
        
        
test_loss /= len(test_loader)

# perplexity
test_perplexity = torch.exp(torch.tensor(test_loss))


# report
print(f"test loss: {test_loss:.4f}, test perplexity: {test_perplexity:.4f}")



epoch 1/1, train loss: 3.3890, train perplexity: 29.6359, val loss: 2.9433, val perplexity: 18.9779
test loss: 2.9484, test perplexity: 19.0747


In [19]:
# %% [markdown]
# #### Implement a text generation function using the trained RNN model. Provide a prompt or seed text, and use the RNN to generate a sequence of characters. Experiment with different prompt texts and observe how the generated text changes. Discuss any interesting patterns or observations you make during the text generation process.
# 


In [20]:



# %%
# with reference to https://github.com/karpathy/char-rnn

def generate_text(model, start_string, length, temperature=1.0):
    
    model.eval()


    # convert characters in to indices and reshape
    input_eval = [char2idx[s] for s in start_string]
    input_eval = torch.tensor(input_eval, dtype=torch.long).unsqueeze(0)
    generated_text = []


    with torch.no_grad():
        
        # iterate for the specified length
        for i in range(length):
            
            output = model(input_eval)
            output = output[:, -1, :] / temperature  
            probabilities = nn.functional.softmax(output, dim=-1)
            
            # sample
            predicted_id = torch.multinomial(probabilities, num_samples=1)
            
            
            # add the text
            input_eval = torch.cat([input_eval, predicted_id], dim=1)

            generated_text.append(idx2char[predicted_id.item()])


    # return text
    return start_string + ''.join(generated_text)


# generate text

# prompts/seeds
prompts = ["hello world", "random sentence be like", "today i wanted to", "please tell me more about"]

for prompt in prompts:
    print(f"prompt: {prompt}")
    print(generate_text(model, prompt, 100))
    
    # divider
    print('-' * 100)



prompt: hello world


NameError: name 'char2idx' is not defined